In [1]:
# current epoch: 20 current mean dice: 0.2194 best mean dice: 0.6352 at epoch 14 for DiceLoss
# current epoch: 20 current mean dice: 0.7359 best mean dice: 0.7712 at epoch 10 for FocalLoss
# current epoch: 20 current mean dice: 0.5530 best mean dice: 0.8096 at epoch 16 for GeneralizedDiceLoss

In [2]:
#!pip install 'monai[all]'

In [3]:
#Thank god-> Starting from Python 3.7, dictionaries maintain the insertion order.

import importlib
import subprocess

def check_and_install_packages(package_list):
    missing_packages = []

    for package in package_list:
        try:
            __import__(package)
        except ImportError:
            missing_packages.append(package)

    if missing_packages:
        print("Installing missing packages:")
        for package in missing_packages:
            subprocess.run(["pip", "install", package])
            print(f"Installed {package}")
    else:
        print("All required packages are already installed.")

# List of packages to check and install if necessary
required_packages = [
    "numpy",
    "matplotlib",
    "pandas",
    #"cv2",
    "imageio",
    "PIL",
    "torch",
    "monai",
    "sys"
]

check_and_install_packages(required_packages)

In [4]:
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import pandas as pd
import os
import cv2 as cv
import imageio
from PIL import Image
import imageio.v3 as iio

In [7]:
#updated_raw_dict, new_analyzed_dict
import PIL

In [8]:
def convert_to_numpy(data):
    if isinstance(data, np.ndarray):
        return data.astype(np.float64)
    elif isinstance(data, PIL.Image.Image):
        return np.array(data).astype(np.float64)
    elif isinstance(data, torch.Tensor):
        return data.numpy().astype(np.float64)
    # Add more conversion cases if needed
    else:
        raise ValueError("Unsupported data format")

def check_and_convert_format(updated_raw_dict, new_analyzed_dict):
    converted_raw_dict = {key: convert_to_numpy(value) for key, value in updated_raw_dict.items()}
    converted_analyzed_dict = {key: convert_to_numpy(value) for key, value in new_analyzed_dict.items()}

    return converted_raw_dict, converted_analyzed_dict


In [9]:
import pickle



In [10]:
import torch
torch.cuda.empty_cache()
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'MAX_SPLIT_SIZE_MB=256'  # Adjust the size as needed

In [11]:
#weight_values = [1.0, 1.5, 2.0, 2.5, 5.0, 10.0, 20.0]  # Example weight values to experiment with

In [12]:
#!pip install tensorboardX
import tensorboardX

In [13]:
import random
# Set a random seed for Python's built-in random module
random_seed = 42
random.seed(random_seed)

# Set a random seed for NumPy
np.random.seed(random_seed)

# Set a random seed for PyTorch on both CPU and CUDA (if available)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [14]:
#!pip install tensorboard

In [15]:
import logging
import os
import sys
import tempfile
from glob import glob

In [16]:
import torch
from torch.cuda.amp import autocast, GradScaler
from torch.utils.tensorboard import SummaryWriter
import torch.nn.utils as nn_utils
import torch.autograd as autograd
from torch.optim.lr_scheduler import StepLR

In [17]:
import psutil

In [18]:
import monai

In [19]:
from monai.data import create_test_image_2d, list_data_collate, decollate_batch, DataLoader

In [20]:
from monai.inferers import sliding_window_inference


In [21]:
from monai.metrics import DiceMetric

In [22]:
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    Compose,
    ScaleIntensityd,
    ToTensor,
    AsDiscrete
)
#   DivisiblePadd deleted

In [23]:
from monai.transforms import Transform
#from monai.visualize import plot_2d_or_3d_image

In [24]:
from monai.networks.layers import Norm
from monai.transforms import Resized

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [26]:
#!pip install scikit-learn

In [27]:
from sklearn.model_selection import train_test_split
import random


In [28]:
from PIL import Image

In [29]:


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_files, segmentation_files, transform=None):
        self.input_files = input_files
        self.segmentation_files = segmentation_files
        self.transform = transform

    def __len__(self):
        return len(self.input_files)

    def __getitem__(self, idx):
        input_path = self.input_files[idx]
        segmentation_path = self.segmentation_files[idx]

        # Load input image (TIFF)
        input_image = Image.open(input_path)
        input_image = np.array(input_image, dtype=np.float32)  # Convert to NumPy array

        # Load segmentation image (TIFF)
        segmentation_image = Image.open(segmentation_path)
        segmentation_image = np.array(segmentation_image, dtype=np.float32)  # Convert to NumPy array

        # Initialize variables
        input_data = input_image
        segmentation_data = segmentation_image

        if self.transform is not None:
            # Apply the specified transforms to input_data and segmentation_data
            transformed_data = self.transform({"img": input_data, "seg": segmentation_data})
            input_data, segmentation_data = transformed_data["img"], transformed_data["seg"]

        return {"img": input_data, "seg": segmentation_data}






max_grad_norm = 1.0
# Define a custom transform to threshold the segmentation tensor
class ThresholdSegmentation(Transform):
    def __init__(self, keys, threshold=0):
        super().__init__()
        self.keys = keys
        self.threshold = threshold

    def __call__(self, data):
        for key in self.keys:
            if key == "seg":
                data[key] = (data[key] > self.threshold).float()
        return data

import matplotlib.pyplot as plt
from monai.transforms import ToNumpy




def main():
    log_dir = "D:/Aytekin/CovidMasterThesis/runs/resize_512_focal_loss_epoch30_16_256_lr1e3_stp3600"
    monai.config.print_config()
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)


    #print(len(updated_raw_dict))
    #print(len(new_analyzed_dict))
    # Check and convert data format only once
    #converted_raw_dict, converted_analyzed_dict = check_and_convert_format(updated_raw_dict, new_analyzed_dict)

    # Remove red for now!!!!
    #converted_raw_dict = {k: v for k, v in converted_raw_dict.items() if 'red' not in k}

    # Remove elements where the key contains "red" for converted_analyzed_dict
    #converted_analyzed_dict = {k: v for k, v in converted_analyzed_dict.items() if 'red' not in k}


    # Check if the dictionaries contain the same length or not, then create train val test:
    #if len(converted_raw_dict) != len(converted_analyzed_dict):
    #    raise ValueError("The lengths of converted_raw_dict and converted_analyzed_dict do not match.")

    #num_images = len(converted_raw_dict)
    #print("Length of converted_raw_dict to be used: {}".format(num_images))
    # Calculate the number of images for training, validation, and test, e.g., using an 80-10-10 split

    #
    #converted_raw_dict.keys()
    
    
    

        # define transforms for image and segmentation
    train_transforms = Compose(
        [
            #ToTensor(),
            EnsureChannelFirstd(keys=["img", "seg"], channel_dim="no_channel"),  # Use channel_dim=-1 for NumPy arrays
            ScaleIntensityd(keys=["img", "seg"]),
            Resized(keys=["img", "seg"],spatial_size=(512, 512), mode="nearest"),
            ThresholdSegmentation(keys=["img", "seg"], threshold=0),

            #DivisiblePadd(keys=["img", "seg"],k=16),

        ]
    )
    val_transforms = Compose(
        [
            #ToTensor(),
            EnsureChannelFirstd(keys=["img", "seg"], channel_dim="no_channel"),  # Use channel_dim=-1 for NumPy arrays
            ScaleIntensityd(keys=["img", "seg"]),
            Resized(keys=["img", "seg"],spatial_size=(512, 512), mode="nearest"),
            ThresholdSegmentation(keys=["img", "seg"], threshold=0),
            #DivisiblePadd(keys=["img", "seg"],k=16),


        ]
    )
    
    
    
    


    # Directory paths for input and segmentation images
    input_image_dir = r"E:\Aytekin\Data\Input"
    segmentation_dir = r"E:\Aytekin\Data\Segmentation"
    test_input_dir = r"E:\Aytekin\Data\Test\Input"
    ignore_dir = r"E:\Aytekin\Data\UniqueIntensityLessThan750"  # New directory to ignore

    # List of strings to exclude from image names
    exclude_strings = ["12_0500", "29_0520", "01_0520", "08_0420", "29_0820", "29_1120", "29_1420", "29_1720", "29_2020", "29_2320"]

    # List all .tif image files in the input and segmentation directories
    input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]
    segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]

    # Get the list of files in the "Test Input" folder
    test_input_files = [os.path.splitext(os.path.basename(filename))[0] for filename in os.listdir(test_input_dir) if filename.endswith(".tif")]

    # Exclude images with the same names as those in the "Test Input" folder
    input_image_files = [filename for filename in input_image_files if os.path.splitext(os.path.basename(filename))[0] not in test_input_files]
    segmentation_files = [filename for filename in segmentation_files if os.path.splitext(os.path.basename(filename))[0] not in test_input_files]

    # Extract file names without extensions
    input_image_names = [os.path.splitext(os.path.basename(filename))[0] for filename in input_image_files]
    segmentation_names = [os.path.splitext(os.path.basename(filename))[0] for filename in segmentation_files]

    # Find the common file names between input and segmentation
    common_names = set(input_image_names) & set(segmentation_names)

    # List all .tif image files in the directories again (overwrite previous lists)
    input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif")]
    segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif")]

    # Exclude images from UniqueIntensityLessThan750 directory
    ignore_files = [os.path.join(ignore_dir, filename) for filename in os.listdir(ignore_dir) if filename.endswith(".tif")]
    ignore_file_names = [os.path.splitext(os.path.basename(filename))[0] for filename in ignore_files]

    # Filter input_image_files and segmentation_files based on common names and ignoring specific names
    input_image_files = [filename for filename in input_image_files if os.path.splitext(os.path.basename(filename))[0] in common_names and os.path.splitext(os.path.basename(filename))[0] not in ignore_file_names]
    segmentation_files = [filename for filename in segmentation_files if os.path.splitext(os.path.basename(filename))[0] in common_names and os.path.splitext(os.path.basename(filename))[0] not in ignore_file_names]



    # Split the data into training and validation sets
    input_image_train, input_image_val, segmentation_train, segmentation_val = train_test_split(
        input_image_files, segmentation_files, test_size=0.2, random_state=42)
    
    # Print the total number of images used in training and validation
    print(f"Total number of images in training set: {len(input_image_train)}")
    print(f"Total number of images in validation set: {len(input_image_val)}")


    # Create instances of the custom dataset for training and validation
    train_dataset = CustomDataset(input_image_train, segmentation_train, transform=train_transforms)
    val_dataset = CustomDataset(input_image_val, segmentation_val, transform=val_transforms)

    # Define data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=0, collate_fn=list_data_collate, pin_memory=torch.cuda.is_available())
    val_loader = DataLoader(val_dataset, batch_size=1, num_workers=0, collate_fn=list_data_collate)







    # define dataset, data loader
    #check_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    # use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
    #check_loader = DataLoader(check_ds, batch_size=2, num_workers=0, collate_fn=list_data_collate)
    #check_data = monai.utils.misc.first(check_loader)
    #print(check_data["img"].shape, check_data["seg"].shape)
    #print("------\n")
    #print(torch.unique(check_data["seg"]))
    # create a training data loader
    #train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    # use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
    #train_loader = DataLoader(
    #    train_ds,
    #    batch_size=2,
    #    shuffle=False,
    #    num_workers=0,
    #    collate_fn=list_data_collate,
    #    pin_memory=torch.cuda.is_available(),
    #)
    # create a validation data loader
    #val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
    #val_loader = DataLoader(val_ds, batch_size=1, num_workers=0, collate_fn=list_data_collate)
    dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
    post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
    # create UNet, DiceLoss and Adam optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    ######## Different Models Can be Tested ##############
    model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=1,
        channels=(16,32,64,128,256),
        strides=(2, 2, 2, 2),
        num_res_units=3,
        norm = Norm.BATCH,
    ).to(device)
    
    
    ######## Different Models Can be Tested ##############
    #model = monai.networks.nets.UNet(
    #    spatial_dims=2,
    #    in_channels=1,
    #    out_channels=1,
    #    channels=(64, 128, 256, 512, 1024, 2048),
    #    strides=(2, 2, 2, 2, 2),
    #    num_res_units=4,
    #    norm=None  # Remove normalization layers
    #).to(device)
    
    loss_function = monai.losses.FocalLoss() #, weight = [1/0.9892240500840984, 1/0.01077594991590156]
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3 ) #1e-3
    #optimizer = Novograd(model.parameters(), lr=1e-2, clip=1.0)
    # start a typical PyTorch training
    val_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = list()
    metric_values = list()
    writer = SummaryWriter(log_dir=log_dir)


    autograd.set_detect_anomaly(True)
    scaler = GradScaler()

    # Define the step size and gamma (decay factor)
    step_size = 3600  # Adjust this according to your training schedule
    gamma = 0.5     # Adjust this according to your preference

    # Create a learning rate scheduler
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    # Create a DiceMetric instance for training
    train_dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)

    for epoch in range(30):
        print("-" * 30)
        print(f"epoch {epoch + 1}/{30}")
        model.train()
        epoch_loss = 0
        step = 0
        # Clear the DiceMetric for the new epoch
        train_dice_metric.reset()
        for batch_data in train_loader:
            step += 1
            inputs, labels = batch_data["img"].to(device), batch_data["seg"].to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(inputs)
                loss = loss_function(outputs, labels)

            # Calculate the Dice score during training
            post_outputs = post_trans(outputs)
            train_dice_metric(y_pred=post_outputs, y=labels)

            scaler.scale(loss).backward()
            #scaler.unscale_(optimizer)  # Unscales the gradients of optimizer's assigned params in-place
            ############## Gradient Clipping Part ###################
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Add this line to clip gradients
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()
            epoch_len = len(train_dataset) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)




        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        # Calculate the average Dice score for the epoch
        train_dice_score = train_dice_metric.aggregate().item()
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}, train_dice: {train_dice_score:.4f}")
        writer.add_scalar("train_dice", train_dice_score, epoch + 1)


        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        scheduler.step()
        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                val_images = None
                val_labels = None
                val_outputs = None
                for val_data in val_loader:
                    val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)
                    roi_size = (96, 96)
                    sw_batch_size = 4
                    val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model)
                    val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]
                    # compute metric for current iteration
                    dice_metric(y_pred=val_outputs, y=val_labels)

                    for i in range(len(val_images)):
                        input_image = val_images[i:i + 1]  # Get the i-th image
                        ground_truth = val_labels[i:i + 1]  # Get the i-th ground truth
                        prediction = val_outputs[i:i + 1]  # Get the i-th prediction

                        # Perform intensity scaling if needed
                        #a_min = 0.0
                        #a_max = 255.0
                        #b_min = 0.0
                        #b_max = 1.0

                        if isinstance(prediction, list):
                            prediction = prediction[0]
                        
                        
                        #input_image_np = input_image.detach().cpu().numpy()  # Convert tensor to NumPy array
                        #ground_truth_np = ground_truth.detach().cpu().numpy()  # Convert tensor to NumPy array
                        #prediction_np = prediction.detach().cpu().numpy()  # Convert tensor to NumPy array
                        
                        #input_image_np = (input_image_np - input_image_np.min()) * (255.0 / (input_image_np.max() - input_image_np.min()))
                        #ground_truth_np = (ground_truth_np - ground_truth_np.min()) * (255.0 / (ground_truth_np.max() - ground_truth_np.min()))
                        #prediction_np = (prediction_np - prediction_np.min()) * (255.0 / (prediction_np.max() - prediction_np.min()))
                        
                        
                        writer.add_image('Validation/Input', input_image, global_step=epoch,dataformats='NCHW')
                        writer.add_image('Validation/GroundTruth', ground_truth, global_step=epoch,dataformats='NCHW')
                        writer.add_image('Validation/Prediction', prediction, global_step=epoch,dataformats='CHW')


                # aggregate the final mean dice result
                metric = dice_metric.aggregate().item()
                # reset the status for next validation round
                dice_metric.reset()
                metric_values.append(metric)
                if metric > best_metric:
                    best_metric = metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), "512green_alldata_focal_loss_epoch50_16_256_lr1e3_stp3600.pth")
                    print("saved new best metric model")
                print(
                    "current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}".format(
                        epoch + 1, metric, best_metric, best_metric_epoch
                    )
                )
                writer.add_scalar("val_mean_dice", metric, epoch + 1)
                # plot the last model output as GIF image in TensorBoard with the corresponding image and label




    autograd.set_detect_anomaly(False)
    writer.close()






if __name__ == "__main__":
    main()

MONAI version: 1.2.0
Numpy version: 1.26.0
Pytorch version: 2.0.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: C:\Users\ge59gor\AppData\Local\anaconda3\envs\deepsight\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: 0.22.0
Pillow version: 9.4.0
Tensorboard version: 2.14.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2
tqdm version: NOT INSTALLED or UNKNOWN VERSION.
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 2.0.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details ab

252/820, train_loss: 0.0656
253/820, train_loss: 0.0633
254/820, train_loss: 0.0640
255/820, train_loss: 0.0640
256/820, train_loss: 0.0623
257/820, train_loss: 0.0613
258/820, train_loss: 0.0610
259/820, train_loss: 0.0594
260/820, train_loss: 0.0591
261/820, train_loss: 0.0578
262/820, train_loss: 0.0582
263/820, train_loss: 0.0557
264/820, train_loss: 0.0548
265/820, train_loss: 0.0574
266/820, train_loss: 0.0548
267/820, train_loss: 0.0543
268/820, train_loss: 0.0526
269/820, train_loss: 0.0515
270/820, train_loss: 0.0504
271/820, train_loss: 0.0511
272/820, train_loss: 0.0494
273/820, train_loss: 0.0498
274/820, train_loss: 0.0485
275/820, train_loss: 0.0482
276/820, train_loss: 0.0467
277/820, train_loss: 0.0460
278/820, train_loss: 0.0464
279/820, train_loss: 0.0446
280/820, train_loss: 0.0448
281/820, train_loss: 0.0447
282/820, train_loss: 0.0440
283/820, train_loss: 0.0430
284/820, train_loss: 0.0411
285/820, train_loss: 0.0415
286/820, train_loss: 0.0432
287/820, train_loss:

545/820, train_loss: 0.0034
546/820, train_loss: 0.0041
547/820, train_loss: 0.0032
548/820, train_loss: 0.0042
549/820, train_loss: 0.0035
550/820, train_loss: 0.0040
551/820, train_loss: 0.0039
552/820, train_loss: 0.0038
553/820, train_loss: 0.0032
554/820, train_loss: 0.0036
555/820, train_loss: 0.0046
556/820, train_loss: 0.0034
557/820, train_loss: 0.0027
558/820, train_loss: 0.0031
559/820, train_loss: 0.0036
560/820, train_loss: 0.0041
561/820, train_loss: 0.0036
562/820, train_loss: 0.0035
563/820, train_loss: 0.0039
564/820, train_loss: 0.0034
565/820, train_loss: 0.0048
566/820, train_loss: 0.0050
567/820, train_loss: 0.0029
568/820, train_loss: 0.0036
569/820, train_loss: 0.0040
570/820, train_loss: 0.0029
571/820, train_loss: 0.0036
572/820, train_loss: 0.0053
573/820, train_loss: 0.0042
574/820, train_loss: 0.0076
575/820, train_loss: 0.0047
576/820, train_loss: 0.0038
577/820, train_loss: 0.0032
578/820, train_loss: 0.0030
579/820, train_loss: 0.0034
580/820, train_loss:

14/820, train_loss: 0.0055
15/820, train_loss: 0.0033
16/820, train_loss: 0.0028
17/820, train_loss: 0.0030
18/820, train_loss: 0.0034
19/820, train_loss: 0.0024
20/820, train_loss: 0.0022
21/820, train_loss: 0.0024
22/820, train_loss: 0.0034
23/820, train_loss: 0.0032
24/820, train_loss: 0.0019
25/820, train_loss: 0.0026
26/820, train_loss: 0.0027
27/820, train_loss: 0.0026
28/820, train_loss: 0.0030
29/820, train_loss: 0.0023
30/820, train_loss: 0.0028
31/820, train_loss: 0.0026
32/820, train_loss: 0.0026
33/820, train_loss: 0.0033
34/820, train_loss: 0.0024
35/820, train_loss: 0.0026
36/820, train_loss: 0.0029
37/820, train_loss: 0.0031
38/820, train_loss: 0.0032
39/820, train_loss: 0.0023
40/820, train_loss: 0.0020
41/820, train_loss: 0.0028
42/820, train_loss: 0.0029
43/820, train_loss: 0.0027
44/820, train_loss: 0.0023
45/820, train_loss: 0.0024
46/820, train_loss: 0.0021
47/820, train_loss: 0.0024
48/820, train_loss: 0.0025
49/820, train_loss: 0.0028
50/820, train_loss: 0.0026
5

310/820, train_loss: 0.0019
311/820, train_loss: 0.0038
312/820, train_loss: 0.0025
313/820, train_loss: 0.0026
314/820, train_loss: 0.0027
315/820, train_loss: 0.0033
316/820, train_loss: 0.0028
317/820, train_loss: 0.0026
318/820, train_loss: 0.0025
319/820, train_loss: 0.0022
320/820, train_loss: 0.0027
321/820, train_loss: 0.0020
322/820, train_loss: 0.0028
323/820, train_loss: 0.0023
324/820, train_loss: 0.0029
325/820, train_loss: 0.0030
326/820, train_loss: 0.0023
327/820, train_loss: 0.0027
328/820, train_loss: 0.0023
329/820, train_loss: 0.0026
330/820, train_loss: 0.0022
331/820, train_loss: 0.0025
332/820, train_loss: 0.0253
333/820, train_loss: 0.0030
334/820, train_loss: 0.0029
335/820, train_loss: 0.0057
336/820, train_loss: 0.0032
337/820, train_loss: 0.0035
338/820, train_loss: 0.0034
339/820, train_loss: 0.0037
340/820, train_loss: 0.0021
341/820, train_loss: 0.0021
342/820, train_loss: 0.0021
343/820, train_loss: 0.0025
344/820, train_loss: 0.0022
345/820, train_loss:

...

367/820, train_loss: 0.0013
368/820, train_loss: 0.0015
369/820, train_loss: 0.0015
370/820, train_loss: 0.0016
371/820, train_loss: 0.0011
372/820, train_loss: 0.0015
373/820, train_loss: 0.0011
374/820, train_loss: 0.0016
375/820, train_loss: 0.0013
376/820, train_loss: 0.0014
377/820, train_loss: 0.0015
378/820, train_loss: 0.0012
379/820, train_loss: 0.0013
380/820, train_loss: 0.0009
381/820, train_loss: 0.0018
382/820, train_loss: 0.0019
383/820, train_loss: 0.0013
384/820, train_loss: 0.0010
385/820, train_loss: 0.0019
386/820, train_loss: 0.0017
387/820, train_loss: 0.0012
388/820, train_loss: 0.0012
389/820, train_loss: 0.0013
390/820, train_loss: 0.0020
391/820, train_loss: 0.0011
392/820, train_loss: 0.0015
393/820, train_loss: 0.0015
394/820, train_loss: 0.0015
395/820, train_loss: 0.0010
396/820, train_loss: 0.0013
397/820, train_loss: 0.0015
398/820, train_loss: 0.0016
399/820, train_loss: 0.0014
400/820, train_loss: 0.0011
401/820, train_loss: 0.0014
402/820, train_loss:

660/820, train_loss: 0.0016
661/820, train_loss: 0.0014
662/820, train_loss: 0.0013
663/820, train_loss: 0.0016
664/820, train_loss: 0.0014
665/820, train_loss: 0.0016
666/820, train_loss: 0.0018
667/820, train_loss: 0.0018
668/820, train_loss: 0.0014
669/820, train_loss: 0.0013
670/820, train_loss: 0.0013
671/820, train_loss: 0.0012
672/820, train_loss: 0.0019
673/820, train_loss: 0.0013
674/820, train_loss: 0.0013
675/820, train_loss: 0.0019
676/820, train_loss: 0.0014
677/820, train_loss: 0.0013
678/820, train_loss: 0.0014
679/820, train_loss: 0.0015
680/820, train_loss: 0.0012
681/820, train_loss: 0.0018
682/820, train_loss: 0.0014
683/820, train_loss: 0.0015
684/820, train_loss: 0.0016
685/820, train_loss: 0.0012
686/820, train_loss: 0.0019
687/820, train_loss: 0.0015
688/820, train_loss: 0.0017
689/820, train_loss: 0.0017
690/820, train_loss: 0.0016
691/820, train_loss: 0.0019
692/820, train_loss: 0.0013
693/820, train_loss: 0.0016
694/820, train_loss: 0.0012
695/820, train_loss:

132/820, train_loss: 0.0012
133/820, train_loss: 0.0014
134/820, train_loss: 0.0013
135/820, train_loss: 0.0017
136/820, train_loss: 0.0017
137/820, train_loss: 0.0011
138/820, train_loss: 0.0013
139/820, train_loss: 0.0013
140/820, train_loss: 0.0013
141/820, train_loss: 0.0015
142/820, train_loss: 0.0013
143/820, train_loss: 0.0014
144/820, train_loss: 0.0012
145/820, train_loss: 0.0017
146/820, train_loss: 0.0013
147/820, train_loss: 0.0020
148/820, train_loss: 0.0012
149/820, train_loss: 0.0012
150/820, train_loss: 0.0019
151/820, train_loss: 0.0018
152/820, train_loss: 0.0019
153/820, train_loss: 0.0015
154/820, train_loss: 0.0016
155/820, train_loss: 0.0015
156/820, train_loss: 0.0018
157/820, train_loss: 0.0013
158/820, train_loss: 0.0013
159/820, train_loss: 0.0015
160/820, train_loss: 0.0019
161/820, train_loss: 0.0010
162/820, train_loss: 0.0016
163/820, train_loss: 0.0014
164/820, train_loss: 0.0011
165/820, train_loss: 0.0015
166/820, train_loss: 0.0015
167/820, train_loss:

425/820, train_loss: 0.0012
426/820, train_loss: 0.0013
427/820, train_loss: 0.0016
428/820, train_loss: 0.0017
429/820, train_loss: 0.0012
430/820, train_loss: 0.0012
431/820, train_loss: 0.0013
432/820, train_loss: 0.0015
433/820, train_loss: 0.0012
434/820, train_loss: 0.0017
435/820, train_loss: 0.0015
436/820, train_loss: 0.0011
437/820, train_loss: 0.0014
438/820, train_loss: 0.0013
439/820, train_loss: 0.0015
440/820, train_loss: 0.0016
441/820, train_loss: 0.0012
442/820, train_loss: 0.0016
443/820, train_loss: 0.0011
444/820, train_loss: 0.0013
445/820, train_loss: 0.0018
446/820, train_loss: 0.0012
447/820, train_loss: 0.0017
448/820, train_loss: 0.0014
449/820, train_loss: 0.0015
450/820, train_loss: 0.0014
451/820, train_loss: 0.0012
452/820, train_loss: 0.0013
453/820, train_loss: 0.0013
454/820, train_loss: 0.0015
455/820, train_loss: 0.0013
456/820, train_loss: 0.0011
457/820, train_loss: 0.0013
458/820, train_loss: 0.0010
459/820, train_loss: 0.0016
460/820, train_loss:

718/820, train_loss: 0.0017
719/820, train_loss: 0.0019
720/820, train_loss: 0.0016
721/820, train_loss: 0.0014
722/820, train_loss: 0.0015
723/820, train_loss: 0.0013
724/820, train_loss: 0.0013
725/820, train_loss: 0.0020
726/820, train_loss: 0.0017
727/820, train_loss: 0.0015
728/820, train_loss: 0.0013
729/820, train_loss: 0.0013
730/820, train_loss: 0.0016
731/820, train_loss: 0.0012
732/820, train_loss: 0.0015
733/820, train_loss: 0.0012
734/820, train_loss: 0.0011
735/820, train_loss: 0.0017
736/820, train_loss: 0.0015
737/820, train_loss: 0.0014
738/820, train_loss: 0.0015
739/820, train_loss: 0.0014
740/820, train_loss: 0.0013
741/820, train_loss: 0.0017
742/820, train_loss: 0.0013
743/820, train_loss: 0.0014
744/820, train_loss: 0.0013
745/820, train_loss: 0.0012
746/820, train_loss: 0.0022
747/820, train_loss: 0.0012
748/820, train_loss: 0.0014
749/820, train_loss: 0.0013
750/820, train_loss: 0.0012
751/820, train_loss: 0.0017
752/820, train_loss: 0.0012
753/820, train_loss:

In [30]:
!tensorboard --logdir=D:/Aytekin/CovidMasterThesis/runs/resize_512_focal_loss_epoch30_16_256_lr1e3_stp3600 --port=6030

^C


In [31]:
print(a)

NameError: name 'a' is not defined

In [ ]:
# Focal loss training starts with a lower loss, so we need smaller learning rate not to get nan values during training.
# That is why 600 is used at step.
###############################################3
# Training started at 9:39 pm 17.12.2023

# import os
#os.system("pkill -f 'tensorboard'")

In [ ]:
#%load_ext tensorboard

In [ ]:
#!pip install --upgrade tensorboard